# Analyse slip velocity particle
In this notebook I analyse the slip velocity (difference fluid velocity and particle velocity)   
to make an estimation for the Reynolds partilcle number ($Re_p = U_{slip} d / \nu $). As  
pointed out by Irina, $Re_p$ might not be small and thus additional corrections to the Stokes  
drag force in the form of $\nu\rightarrow\nu_{eff}=\nu c(Re_p) = \nu(1+f(RE_p))$ should be added.       
To investigate this we measure the slip velocity in simulations with MR advection using:  
1. stokes drag
2. drag with  $c(Re_p)$ dependend drag   
And we do this for the slow manifold and full MR equation. We also investigate:   
1. variability $Re_p$
2. spin up time simulation (how mcuh time is needed before  slip velocity / $Re_p$ stabilizes)
32. effect of different timesteps


In [ ]:
# import needed packages

#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from decimal import Decimal
from scipy.optimize import fsolve
from matplotlib import colormaps

from datetime import datetime, timedelta
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, make_lognormal_PDF
import particle_characteristics_functions as pcf

sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
from helper import create_filelist

plt.style.use('../python_style_Meike.mplstyle')
# plotstyle:
color_array = np.array(['c','orange','purple','black'])
marker_array = np.array(['s','o','>','p','^','D'])
line_array = np.array(['-','--','-.',':'])
markerline_array = np.array(['--s','--o','-->','--p','--^','--D'])
Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.0, 0.9, 6))


base_directory = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/test_history_term/')

basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')


particle_types = ['inertial_SM_Rep_constant','inertial_Rep_constant','inertial_SM_drag_Rep','inertial_drag_Rep']

file_names = {'inertial_SM_Rep_constant': basefile_Rep_constant,
              'inertial_Rep_constant':basefile_Rep_constant,
              'inertial_SM_drag_Rep': basefile_Rep_drag,
              'inertial_drag_Rep': basefile_Rep_drag}



In [ ]:
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water

In [ ]:
#compare drag factors

fig,ax = plt.subplots()
rep = np.logspace(-1,3,1000)
ax.plot(rep,pcf.factor_drag_morrison2013(rep),'-',color='k')
ax.plot(rep,pcf.factor_drag_white1991(rep),'--',color='green')
# ax.plot(rep,1+0.095*rep**0.8,'--',color='red')
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_xlabel('Re$_p$')
# ax.set_ylabel('c(Re$_p$)')
ax.set_ylabel('$\\nu_{\\mathrm{eff}}/\\nu$')
ax.legend(['Morrison 2013','White 1991'])

print(pcf.factor_drag_white1991(457))
# we use white 1991 throughout as it is a way simpler expression and still does a very good job up to 10^5

# constant reynolds number
We perform simulation using drag force with $c(\mathrm{Re}_p)$ and measure the slip velocity  
we use this slip veloctity to calculated the measured $\mathrm{Re}_p$  
we compare the input $\mathrm{Re}_p$ with the measured $\mathrm{Re}_p$. The goal is to find  
the Reynolds number for which the input and measured $\mathrm{Re}_p$ are the same.  
We first use a short simulation of 24 hours where we save the slip velocity every 5 min and  
advect the particles using the slow manifold MR equation (more stable for high Reynolds numbers)  


## 24 hours sim with slow manifold

In [ ]:
# read in data
Replist = np.array([0,200,400,450,600,800,1000])


#settings
runtime =  timedelta(days=2)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2994.76# 2759.97
coriolis = True
gradient = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
endtime = starttime + runtime
particle_type = 'inertial_SM_Rep_constant'

chunck_time =100
nparticles = 52511
data = {}
for pt in particle_types:
    data[pt]={}

for pt in particle_types:
    print(pt)
    if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
        for Rep in Replist: 
            file = file_names[pt].format(loc=loc,
                                        particle_type = pt,
                                        y_s=starttime.year,
                                        m_s=starttime.month,
                                        d_s=starttime.day,
                                        y_e=endtime.year,
                                        m_e=endtime.month,
                                        d_e=endtime.day,
                                        B = int(B * 1000), 
                                        tau = int(tau ),
                                        land_handling = land_handling, 
                                        coriolis = coriolis,
                                        Rep = Rep ,
                                        gradient = gradient)
            ds = xr.open_dataset(file,
                                            engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['B','tau','z'],
                                            decode_times=False) #,decode_cf=False)
            Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
            ds = ds.assign(Uslip = Uslip)
            Rep_measured = pcf.Re_particle(Uslip,diameter, kinematic_viscosity_water)
            ds = ds.assign(Rep_measured = Rep_measured)
            ds.Uslip[:,0]=np.nan
            data[pt][Rep]=ds
    else:

        file = file_names[pt].format(loc=loc,
                                    particle_type = pt,
                                    y_s=starttime.year,
                                    m_s=starttime.month,
                                    d_s=starttime.day,
                                    y_e=endtime.year,
                                    m_e=endtime.month,
                                    d_e=endtime.day,
                                    B = int(B * 1000), 
                                    tau = int(tau ),
                                    land_handling = land_handling, 
                                    coriolis = coriolis,
                                    Rep = 0 ,
                                    gradient = gradient)
        print(file)
        ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)
        Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
        ds = ds.assign(Uslip = Uslip)
        Rep_measured = pcf.Re_particle(Uslip,diameter, kinematic_viscosity_water)
        ds = ds.assign(Rep_measured = Rep_measured)
        ds.Uslip[:,0]=np.nan
        data[pt][None]=ds

In [ ]:
# plot selection of slip velocities
fig, ax = plt.subplots()
i=0
id=152#2#25#17# #17# 14 #14
Replist_selection = [200,450,600,800,1000]
legend = []
pt = 'inertial_Rep_constant'
print(f'start loc ({id}) = ({data[pt][0].lon[id,0].values:.02f},{data[pt][0].lat[id,0].values:.02f})')

for Rep, color in zip(Replist_selection, Magmalist):
    ax.plot((data[pt][Rep].time[id,1:]-data[pt][Rep].time[id,0])/sec_in_hours,data[pt][Rep].Rep_measured[id,1:],'--o',color=color)#,marker_array[i],color=color_array[i])   
    legend.append('$\\mathrm{Re_p} =$' +  f'{Rep}')
    i+=1

ax.legend(legend)
ax.set_xlabel('time [hours]')
ax.set_ylabel('zonal slip velocity [m/s]')


fig2, ax2= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax2.coastlines()


scp = ax2.plot(data[pt][0].lon[id,:].values,data[pt][0].lat[id,:].values.T)
ax2.set_xlim(-16,10)
ax2.set_ylim(46,61.3)


In [ ]:
data[pt][Rep].time[id,1:]

In [ ]:
# make PDF and CDF for every particle Reynolds number
mean_SM_Rep_c_day={}
bins_SM_Rep_c_day={}
pdfs_SM_Rep_c_day={}
cdfs_SM_Rep_c_day={}
pt = 'inertial_SM_Rep_constant'

for Rep in Replist:
    Re_p =pcf.Re_particle(data[pt][Rep].Uslip[:,6:],diameter, kinematic_viscosity_water)

    array = Re_p.values.flatten()
    array=array[~np.isnan(array)]
  
    mean_SM_Rep_c_day[Rep]=np.mean(array,)
    
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_SM_Rep_c_day[Rep]=bins
    pdfs_SM_Rep_c_day[Rep]=pdf
    cdfs_SM_Rep_c_day[Rep]=cdf

In [ ]:
# make PDF for every particle Reynolds number
mean={}
for pt in particle_types:
    mean[pt]={}

pt = 'inertial_SM_Rep_constant'
fig, ax = plt.subplots()
for pt, marker in zip(['inertial_SM_Rep_constant','inertial_Rep_constant'],['-','--']):
    print(pt)
    for Rep in Replist:
        array = data[pt][Rep].Rep_measured[:,6:].values.flatten()
        array=array[~np.isnan(array)]
        print(np.max(array))
        mean[pt][Rep]=np.mean(array)
        median=np.median(array)
        # print(median)
       
    
        bins,pdf = make_PDF(array,nbins = 10001, norm = True,vmin=1,vmax=10000)#,min=0,max=750000)
        ax.plot(bins,pdf)

ax.legend(np.char.add('Re$_p$ = ',  Replist.astype(str)))
ax.set_xscale('log')
# ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')


Rep = None
pt = 'inertial_SM_drag_Rep'
array = data[pt][Rep].Rep_measured[:,6:].values.flatten()
array=array[~np.isnan(array)]
print(np.max(array))
mean[pt][Rep]=np.mean(array)
print(mean[pt][Rep])
bins,pdf = make_PDF(array,nbins = 10001, norm = True,vmin=1,vmax=10000)#,min=0,max=750000)
ax.plot(bins,pdf,'-', color='navy')


Rep = None
pt = 'inertial_drag_Rep'
array = data[pt][Rep].Rep_measured[:,6:].values.flatten()
array=array[~np.isnan(array)]
print(np.max(array))
array = array[array<1000000]
mean[pt][Rep]=np.mean(array)
print(mean[pt][Rep])
bins,pdf = make_PDF(array,nbins = 10001, norm = True,vmin=1,vmax=10000)#,min=0,max=750000)
ax.plot(bins,pdf,'--', color='firebrick')




In [ ]:
pcf.factor_drag_white1991(10E5)*1/(2997) * 60


In [ ]:
fig, ax  = plt.subplots()
Rep = None
pt = 'inertial_drag_Rep'
array = data[pt][Rep].Rep_measured[:,6:].values.flatten()
array=array[array<5000]
mean[pt][Rep]=np.mean(array)
print(np.mean(array))
bins,pdf = make_PDF(array,nbins = 10001, norm = True,vmin=1,vmax=10000)#,min=0,max=750000)
ax.plot(bins,pdf,'--', color='firebrick')
ax.set_xscale('log')
# ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
test = np.sort(array)
print(test[-1000:])


In [ ]:
# plot pdfs
fig, ax = plt.subplots()

for Rep, color, marker in zip(Replist, Magmalist, marker_array):
   ax.plot(bins_SM_Rep_c_day[Rep],pdfs_SM_Rep_c_day[Rep],marker,color=color)
ax.legend(np.char.add('Re$_p$ = ',  Replist.astype(str)))
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
for Rep, color, marker in zip(Replist_selection, color_array, line_array):
   
   ax.plot(bins_SM_Rep_c_day[Rep],cdfs_SM_Rep_c_day[Rep],marker,color=color)
   i+=1 

ax.legend(np.char.add('Re$_p$ = ', Replist_selection.astype(str)))
ax.set_xscale('log')

ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

### Estimate when measured and input particle reynolds number are the same
To estimate when the measured and input reynolds number are the same we use the  
fact that the slip velocity scales with $C(\mathrm{Re}_p^{\mathrm{in}})$, Thus we try   
to predict $\mathrm{Re}_p^{\mathrm{m}}$ using a single reference measurement with   
$\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{in}}$ resulting in $\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{m}}$  
Then we calculate the  $\mathrm{Re}_p^{\mathrm{measured}}$ using:   
$$  \mathrm{Re}_p^{\mathrm{m}}=\frac{\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{m}}  C(\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{in}})}{C(\mathrm{Re}_{p}^{\mathrm{in}}) }  $$

In [ ]:
# define functions needed to estimate the measured Rep based on a single reference measured
def Rep_measured(Rep_in, Rep_ref_in, Rep_ref_measured):
    return Rep_ref_measured * pcf.factor_drag_white1991(Rep_ref_in)/pcf.factor_drag_white1991(Rep_in)

def Rep_in_measured_root(Rep_in, Rep_ref_in, Rep_ref_measured):
   return Rep_measured(Rep_in, Rep_ref_in, Rep_ref_measured)-Rep_in

In [ ]:
mean['inertial_Rep_constant']

In [ ]:
# 

# Repmeasured_SM = [mean['inertial_SM_Rep_constant'][Rep] for Rep in Replist]
# solution_SM = [fsolve(Rep_in_measured_root, 450, args=(Repi, Repm))[0] for Repi,Repm in zip(Replist,Repmeasured_SM)]
# Rep_SM_fit =  np.array(solution_SM[1:]).mean()

#plotting
fig, ax = plt.subplots()
# # plot measured points
# ax.plot(Replist,Repmeasured_SM,'o',color='navy')
# # plot predicted solution based on single value 
Reprange = np.logspace(-1,4,1000)
# ax.plot(Reprange,Repmeasured_SM[2] * pcf.factor_drag_white1991(Replist[2])/pcf.factor_drag_white1991(Reprange),'--',zorder=-5,color='navy')
# #legend
# ax.plot(Rep_SM_fit,Rep_SM_fit,'o',color='grey')
# # plot fitted solution'

# ax.plot(mean['inertial_SM_drag_Rep'][None],mean['inertial_SM_drag_Rep'][None],'o',color='k')
Replist = [0,200,400,600,800,1000]

Repmeasured = [mean['inertial_Rep_constant'][Rep] for Rep in Replist]
solution = [fsolve(Rep_in_measured_root, 450, args=(Repi, Repm))[0] for Repi,Repm in zip(Replist,Repmeasured)]
Rep_fit =  np.array(solution[1:]).mean()
ax.plot(Replist,Repmeasured,'s',color='firebrick',zorder=-2)
ax.plot(Reprange,Repmeasured[2] * pcf.factor_drag_white1991(Replist[2])/pcf.factor_drag_white1991(Reprange),'--',zorder=-5,color='firebrick')
ax.plot(Rep_fit,Rep_fit,'o',color='grey')
ax.plot(mean['inertial_drag_Rep'][None],mean['inertial_drag_Rep'][None],'D',color='cornflowerblue')
# ax.legend(['SM: measured Re$_p^{\\mathrm{M}}$','SM: predicted Re$_p^{\\mathrm{M}}$','SM: Re$_p^{\\mathrm{In}}$ = Re$_p^{\\mathrm{M}}$ = '+f'{Rep_SM_fit:.0f}','SM: flexible Re$_p$ = '+f'{mean['inertial_SM_drag_Rep'][None]:.0f}','full: measured Re$_p^{\\mathrm{M}}$','full: predicted Re$_p^{\\mathrm{M}}$','full: Re$_p^{\\mathrm{In}}$ = Re$_p^{\\mathrm{M}}$ = '+f'{Rep_fit:.0f}','full: flexible Re$_p$ = '+f'{mean['inertial_drag_Rep'][None]:.0f}'], loc =1,fontsize=18, frameon=True)

ax.legend(['measured Re$_p^{\\mathrm{M}}$','predicted Re$_p^{\\mathrm{M}}$','Re$_p^{\\mathrm{In}}$ = Re$_p^{\\mathrm{M}}$ = '+f'{Rep_fit:.0f}','flexible $\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$ = '+f'{mean['inertial_drag_Rep'][None]:.0f}'],loc =1,fontsize=18, frameon=True)

ax.plot(Reprange,Reprange,color='lightgrey',zorder=-10)
# ax.text(85,60,'Re$_p^{in}$ = Re$_p^m$ = '+f'{Rep_fit:.2f}',color='grey',fontsize=15)
# label layout
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.set_xlim(1E1,1E4)
# ax.set_ylim(1E1,1E4)
ax.set_xlim(0,1200)
ax.set_ylim(0,1200)
# ax.set_ylim(1E1,1E4)
ax.set_xlabel('Re$_p^{\\mathrm{In}}$')
ax.set_ylabel('$\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$')

fig.tight_layout()
fig.savefig('../figures/sensitivity_tests/full_MR_Rep_measured_vs_Rep_in.pdf')


# #print solution
# print(f'Rep fit = {Rep_fit:.2f}')



In [ ]:
Repmeasured[0]

In [ ]:
print(solution)

In [ ]:
# plot selection of slip velocities
fig, ax = plt.subplots()#figsize=(10,5))
i=0
id=66#4#45#2 #14 #6754 #25#17# #17# 14 #14
# print
legend = []
# pt = 'inertial_SM_Rep_constant'
# ax.plot((data[pt][600].time[id,1:]-data[pt][400].time[id,0])/sec_in_hours/24,data[pt][400].Rep_measured[id,1:],'-->',color='purple')#,marker_array[i],color=color_array[i])   
# pt = 'inertial_SM_drag_Rep'
# ax.plot((data[pt][None].time[id,1:]-data[pt][None].time[id,0])/sec_in_hours/24,data[pt][None].Rep_measured[id,1:],'-o',color='navy')#,marker_array[i],color=color_array[i])   


pt = 'inertial_Rep_constant'
for Rep,color,marker in zip([200,400,600],Magmalist[1::2],marker_array):
    ax.plot((data[pt][Rep].time[id,1::12]-data[pt][Rep].time[id,0])/sec_in_hours,data[pt][Rep].Rep_measured[id,1::12],'-.o',color=color)#,marker_array[i],color=color_array[i])   
pt = 'inertial_drag_Rep'

ax.plot((data[pt][None].time[id,1::12]-data[pt][None].time[id,0])/sec_in_hours,data[pt][None].Rep_measured[id,1::12],'--s',color='cornflowerblue')#,marker_array[i],color=color_array[i])   


pt = 'inertial_SM_Rep_constant'
ax.set_title(f'start location = ({data[pt][0].lon[id,0].values:.02f}'+'$^{\\circ}$'+f', {data[pt][0].lat[id,0].values:.02f}'+'$^{\\circ}$)',fontsize=18)

ax.legend(['$\\mathrm{Re}_p^{\\mathrm{In}}$ = 200', '$\\mathrm{Re}_p^{\\mathrm{In}}$ = 400', '$\\mathrm{Re}_p^{\\mathrm{In}}$ = 600','flexible $\\mathrm{Re}_p$'],fontsize=18)

ax.set_xlabel('time [hours]')
ax.set_ylabel('Re$_p^{\\mathrm{M}}$ ')
# ax.set_ylim(0,401)




In [ ]:
# plot selection of slip velocities
fig, ax = plt.subplots(figsize=(10,5))
i=0
id=152 #14 #6754 #25#17# #17# 14 #14
# print
legend = []
pt = 'inertial_SM_Rep_constant'
ax.plot((data[pt][600].time[id,1:]-data[pt][400].time[id,0])/sec_in_hours/24,data[pt][400].Rep_measured[id,1:],'-->',color='purple')#,marker_array[i],color=color_array[i])   
pt = 'inertial_SM_drag_Rep'
ax.plot((data[pt][None].time[id,1:]-data[pt][None].time[id,0])/sec_in_hours/24,data[pt][None].Rep_measured[id,1:],'-o',color='navy')#,marker_array[i],color=color_array[i])   
pt = 'inertial_Rep_constant'
ax.plot((data[pt][600].time[id,1:]-data[pt][400].time[id,0])/sec_in_hours/24,data[pt][400].Rep_measured[id,1:],'--^',color='orange')#,marker_array[i],color=color_array[i])   
pt = 'inertial_drag_Rep'

ax.plot((data[pt][None].time[id,1:]-data[pt][None].time[id,0])/sec_in_hours/24,data[pt][None].Rep_measured[id,1:],'-s',color='firebrick')#,marker_array[i],color=color_array[i])   


pt = 'inertial_SM_Rep_constant'
ax.set_title(f'start location = ({data[pt][0].lon[id,0].values:.02f}'+'$^{\\circ}$'+f', {data[pt][0].lat[id,0].values:.02f}'+'$^{\\circ}$)')

ax.legend(['SM: Re$_p^{\\mathrm{In}}$ = 400','SM: Re$_p$ flexible', 'full: Re$_p^{\\mathrm{In}}$ = 400','full: Re$_p$ flexible'],loc=(1,0.2),fontsize=18)

ax.set_xlabel('time [days]')
ax.set_ylabel('Re$_p^{\\mathrm{M}}$ ')
ax.set_ylim(0,1500)




In [ ]:
# plot selection of slip velocities
fig, ax = plt.subplots(figsize=(10,5))
i=0
id=2 #14 #6754 #25#17# #17# 14 #14
# print
legend = []
Rep=None#400
pt = 'inertial_drag_Rep'
# pt = 'inertial_SM_Rep_constant'
id=2
ax.plot((data[pt][Rep].time[id,1:]-data[pt][Rep].time[id,0])/sec_in_hours/24,data[pt][Rep].Rep_measured[id,1:],'-s',color='black')#,marker_array[i],color=color_array[i])   
legend.append(f'({data[pt][Rep].lon[id,0].values:.02f}'+'$^{\\circ}$'+f', {data[pt][Rep].lat[id,0].values:.02f}'+'$^{\\circ}$)')
id=152
ax.plot((data[pt][Rep].time[id,1:]-data[pt][Rep].time[id,0])/sec_in_hours/24,data[pt][Rep].Rep_measured[id,1:],'--o',color='grey')#,marker_array[i],color=color_array[i])   
legend.append(f'({data[pt][Rep].lon[id,0].values:.02f}'+'$^{\\circ}$'+f', {data[pt][Rep].lat[id,0].values:.02f}'+'$^{\\circ}$)')
# pt = 'inertial_SM_Rep_constant'


ax.legend(legend)#,loc=(1,0.3))

ax.set_xlabel('time [days]')
ax.set_ylabel('Re$_p^{\\mathrm{M}}$ ')
ax.set_ylim(0,1001)

fig.tight_layout()
fig.savefig('../figures/sensitivity_tests/Rep_measured_2_locations_over_time_full_MR_Rep_flexible.pdf')




In [ ]:
id = 150
pt = 'inertial_SM_Rep_constant'
print(f'start location = ({data[pt][0].lon[id,0].values:.02f}'+'$^{\\circ}$'+f', {data[pt][0].lat[id,0].values:.02f}'+'$^{\\circ}$)')

In [ ]:
fig, ax = plt.subplots()
i=0
id=20 #25#17# #17# 14 #14
Rep = 400
legend = []
pt = 'inertial_Rep_constant'
ax.plot((data[pt][Rep].time[id,1:]-data[pt][Rep].time[id,0])/sec_in_hours/24,data[pt][Rep].Rep_measured[id,1:],'--s',color='firebrick')#,marker_array[i],color=color_array[i])   
pt = 'inertial_SM_Rep_constant'
ax.plot((data[pt][Rep].time[id,1:]-data[pt][Rep].time[id,0])/sec_in_hours/24,data[pt][Rep].Rep_measured[id,1:],'--o',color='navy')#,marker_array[i],color=color_array[i])   

ax.legend(legend)
ax.set_xlabel('time [days]')
ax.set_ylabel('Re$_p$ ')

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_drag_Rep'
tid =24*12#13 # 24


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
sec = float(data[pt][None].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title('flexible Re$_p$, ' + f'{date}', fontsize=15)
scp = ax.scatter(data[pt][None].lon[::1,tid].T,data[pt][None].lat[::1,tid].T,c=data[pt][None].Rep_measured[::1,tid].T,s=0.4, vmin=0,vmax=2000,cmap='magma')
fig.colorbar(scp,label='Re$_p^{\\mathrm{M}}$',fraction=0.03,extend='max')
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0.1, color='gray', alpha=0, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

fig.tight_layout()
# fig.savefig('../figures/sensitivity_tests/scatter_plot_measured_Rep_flexible_rep_24h.pdf')

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_Rep_constant'
tid =24*12#13 # 24
Rep = 450


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
sec = float(data[pt][Rep].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title('Re$_p^{\\mathrm{In}}=450$, ' + f'{date}', fontsize=15)
scp = ax.scatter(data[pt][Rep].lon[::1,tid].T,data[pt][Rep].lat[::1,tid].T,c=data[pt][Rep].Rep_measured[::1,tid].T,s=0.4, vmin=0,vmax=2000,cmap='magma')
fig.colorbar(scp,label='Re$_p^{\\mathrm{M}}$',fraction=0.03,extend='max')
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0.1, color='gray', alpha=0, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

fig.tight_layout()

## 24 hours sim full MR 
To check that the SM and full MR agree we also performed full MR simulations for a supset or the Rep input values used in the SM MR sim

In [ ]:
#import data
Replist = np.array([300,500, 1000, 5000])
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')
#settings
runtime =  timedelta(days=1)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 2, 0, 0, 0, 0)
endtime = starttime + runtime
particle_type = 'inertial_Rep_constant'

chunck_time = 100
nparticles = 88347
data_Rep_constant_24h = {}
for Rep in Replist: 
    file = file_base.format(loc=loc,
                           particle_type = particle_type,
                                                            y_s=starttime.year,
                                                            m_s=starttime.month,
                                                            d_s=starttime.day,
                                                            y_e=endtime.year,
                                                            m_e=endtime.month,
                                                            d_e=endtime.day,
                                                            B = int(B * 1000), 
                                                            tau = int(tau ),
                                                            land_handling = land_handling, 
                                                            cor_on = coriolis,
                                                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_constant_24h[Rep]=ds

In [ ]:
kernels_init = ['test 1 ','test 2']
kernels=['test 4', 'djsadkjfa', '333']
kernels_init = kernels_init + kernels
print(kernels_init)

In [ ]:
id=1
fig, ax = plt.subplots()
i=0


for Rep, color, marker in zip(Replist,color_array,marker_array):
    ax.plot((data_Rep_constant_24h[Rep].time[id]-data_Rep_constant_24h[Rep].time[id,0])*sec_in_hours,data_Rep_constant_24h[Rep].uf[id],marker,color=color)   


ax.legend(np.char.add('Re$_p$ = ',  Replist.astype(str)))
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [hours]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_c_day={}
bins_Rep_c_day={}
pdfs_Rep_c_day={}
cdfs_Rep_c_day={}

for Rep in Replist:
    Re_p = Re_particle(data_Rep_constant_24h[Rep].Uslip,diameter, kinematic_viscosity_water)
    array = Re_p.values.flatten()
    array=array[~np.isnan(array)]

    mean_Rep_c_day[Rep]=np.mean(array)
    
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_c_day[Rep]=bins
    pdfs_Rep_c_day[Rep]=pdf
    cdfs_Rep_c_day[Rep]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for Rep, color, marker in zip(Replist[0:4], color_array, marker_array):
   ax.plot(bins_Rep_c_day[Rep],pdfs_Rep_c_day[Rep],marker,color=color)
ax.legend(np.char.add('Re$_p$ = ',  Replist[0:4].astype(str)))
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
i=0
for Rep in Replist[0:4]:
   
   ax.plot(bins_Rep_c_day[Rep],cdfs_Rep_c_day[Rep],line_array[i],color=color_array[i])
   i+=1 

ax.legend(np.char.add('Re$_p$ = ',  Replist[0:4].astype(str)))
ax.set_xscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# compare with fit and measured values
Repmeasured = [mean_Rep_c_day[Rep] for Rep in Replist]
#plotting
fig, ax = plt.subplots()
# plot measured points

ax.plot(Replist,Repmeasured,'s',color='firebrick')
ax.plot(Replist_SM,Repmeasured_SM,'o',color='navy')
# plot predicted solution based on single value 
Reprange = np.logspace(-1,4,1000)
ax.plot(Reprange,Repmeasured_SM[5] * factor_drag_white1991(Replist_SM[5])/factor_drag_white1991(Reprange),'--',zorder=-5,color='navy')
#legend
ax.legend(['simulated Re$_p^m$ ','simulated Re$_p^m$ SM','predicted Re$_p^m$'])
# plot fitted solution
ax.plot(Reprange,Reprange,color='grey',zorder=-10)
ax.text(85,60,'Re$_p^{in}$ = Re$_p^m$ = '+f'{Rep_fit:.2f}',color='grey',fontsize=15)
# label layout
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(0.1,10**4)
ax.set_ylim(0.1,10**4)
ax.set_xlabel('Re$_p^{in}$')
ax.set_ylabel('Re$_p^{m}$')
# does agree :)

# 24 h  simulation with timevarying drag coefficient
Try time varying drag coefficient with SM MR and full MR 

In [ ]:
# import data
particle_types = ['inertial_SM_drag_Rep','inertial_drag_Rep']
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')
#settings
runtime =  timedelta(days=1)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 2, 0, 0, 0, 0)
endtime = starttime + runtime
particle_type = 'inertial_Rep_constant'
Rep = 457

chunck_time = 100
nparticles = 88347
data_Rep_var_24h = {}
for particle_type in particle_types: 
    file = file_base.format(loc=loc,
                           particle_type = particle_type,
                                                            y_s=starttime.year,
                                                            m_s=starttime.month,
                                                            d_s=starttime.day,
                                                            y_e=endtime.year,
                                                            m_e=endtime.month,
                                                            d_e=endtime.day,
                                                            B = int(B * 1000), 
                                                            tau = int(tau ),
                                                            land_handling = land_handling, 
                                                            cor_on = coriolis,
                                                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_var_24h[particle_type]=ds


In [ ]:

id=1
fig, ax = plt.subplots()

for particle_type, color, marker in zip(particle_types,color_array,marker_array):
    ax.plot((data_Rep_var_24h[particle_type].time[id]-data_Rep_var_24h[particle_type].time[id,0])/sec_in_hours,data_Rep_var_24h[particle_type].uf[id],marker,color=color)   


ax.legend(particle_types)
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [hours]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_var_day={}
bins_Rep_var_day={}
pdfs_Rep_var_day={}
cdfs_Rep_var_day={}

for pt in particle_types:
    Re_p = Re_particle(data_Rep_var_24h[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_p.values.flatten()
    array=array[~np.isnan(array)]
    array=array[array<100000]

    mean_Rep_var_day[pt]=np.mean(array)
    
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_var_day[pt]=bins
    pdfs_Rep_var_day[pt]=pdf
    cdfs_Rep_var_day[pt]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,marker_array,color_array):
   
   ax.plot(bins_Rep_var_day[pt],pdfs_Rep_var_day[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

# full MR unstable for some partciles s

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,line_array,color_array):
   
   ax.plot(bins_Rep_var_day[pt],cdfs_Rep_var_day[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')

ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

In [ ]:
for pt in particle_types:
    print(mean_Rep_var_day[pt])
    print(factor_drag_morrison2013(mean_Rep_var_day[pt]))

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_SM_drag_Rep'
Re_p = Re_particle(data_Rep_var_24h[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
tid = 10


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
sec = float(data_Rep_var_24h[pt].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title(f'{date}', fontsize=15)
scp = ax.scatter(data_Rep_var_24h[pt].lon[::1,tid].T,data_Rep_var_24h[pt].lat[::1,tid].T,c=c_rep[::1,tid].T,s=0.3, vmin=0,vmax=30,cmap='magma')
fig.colorbar(scp,label='$\\nu_{\\mathrm{eff}}/\\nu$',fraction=0.03)
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)

In [ ]:
# make movie out of it
pt = 'inertial_drag_Rep'
Re_p = Re_particle(data_Rep_var_24h[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
for tid in range(1,288,1):

    print(tid)

    fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    sec = float(data_Rep_var_24h[pt].time[0,tid].values)
    date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
    ax.set_title(f'{date}', fontsize=15)
    ax.coastlines()
    ax.set_xlim(-16,10)
    ax.set_ylim(46,61.3)
    scp = ax.scatter(data_Rep_var_24h[pt].lon[::1,tid].T,data_Rep_var_24h[pt].lat[::1,tid].T,c=c_rep[::1,tid].T,s=0.3, vmin=0,vmax=20)
    fig.colorbar(scp,label='$Re_p$',fraction=0.03)
    fig.tight_layout()
    fig.savefig(f'/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/figures/lagrangian_field/frames/september/Re_particle/24_h_hourly_field/crep/frames{tid:04d}.png')
    plt.close(fig)
# ffmpeg -framerate 20 -i frames%04d.png  -vcodec libx264 -pix_fmt yuv420p -r 10 scatter_MR_c_rep_drag.mp4
# ffmpeg -framerate 20 -i frames%04d.png  -vcodec libx264 -pix_fmt yuv420p -r 10 scatter_MRSM_hourly_field_c_rep_drag.mp4
# ffmpeg -framerate 20 -i frames%04d.png  -vcodec libx264 -pix_fmt yuv420p -r 10 scatter_MRSM_daily_field_c_rep_drag.mp4



In [ ]:
data_Rep_var_24h[pt].lon.shape

## 1 month simulations constant Rep
1 month sim with both full MR and slow manifold using $\mathrm{Re}_p^{\mathrm{in}} = 457$
and hourly output


In [ ]:
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}_Rep_{Rep:04d}_slip_velocity.zarr')

#settings
runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
endtime = starttime + runtime
particle_types = ['inertial_SM_Rep_constant','inertial_Rep_constant']
data_Rep_constant_1month={}
chunck_time = 100
nparticles = 88347
Rep = 457
for pt in particle_types: 
    file = file_base.format(loc=loc,
                           particle_type = pt,
                            y_s=starttime.year,
                            m_s=starttime.month,
                            d_s=starttime.day,
                            y_e=endtime.year,
                            m_e=endtime.month,
                            d_e=endtime.day,
                            B = int(B * 1000), 
                            tau = int(tau ),
                            land_handling = land_handling, 
                            cor_on = coriolis,
                            Rep = Rep )
    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)
    Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_constant_1month[pt]=ds

In [ ]:

id=1
fig, ax = plt.subplots()

for pt, color, marker in zip(particle_types,color_array,marker_array):
    ax.plot((data_Rep_constant_1month[pt].time[id]-data_Rep_constant_1month[pt].time[id,0])/sec_in_hours/24,data_Rep_constant_1month[pt].uf[id],marker,color=color)   


ax.legend(particle_types)
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [days]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_c_month={}
bins_Rep_c_month={}
pdfs_Rep_c_month={}
cdfs_Rep_c_month={}
mean_Rep_c_over_month={}

for pt in particle_types:
    Re_p = Re_particle(data_Rep_constant_1month[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_p.values.flatten()
    array=array[~np.isnan(array)]
    # array=array[array<100000]

    mean_Rep_c_month[pt]=np.mean(array)
    mean_Rep_c_over_month[pt]=Re_p.mean(dim='trajectory',skipna=True)
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_c_month[pt]=bins
    pdfs_Rep_c_month[pt]=pdf
    cdfs_Rep_c_month[pt]=cdf

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,marker_array,color_array):
   
   ax.plot(bins_Rep_c_month[pt],pdfs_Rep_c_month[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot cdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,line_array,color_array):
   print(mean_Rep_c_month[pt])
   ax.plot(bins_Rep_c_month[pt],cdfs_Rep_c_month[pt],marker,color=color)

ax.legend(particle_types)
ax.set_xscale('log')


ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')


In [ ]:
fig, ax = plt.subplots()
id=1
for pt, marker, color in zip(particle_types,marker_array,color_array):
    ax.plot((data_Rep_constant_1month[pt].time[id]-data_Rep_constant_1month[pt].time[id,0])/sec_in_hours/24,mean_Rep_c_over_month[pt], marker, color=color )

# 1 month varying Rep

In [ ]:
file_base = ('/storage/shared/oceanparcels/output_data/'
             'data_Meike/MR_advection/NWES/{particle_type}/'
            '{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{cor_on}.zarr')

#settings
#  /storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_SM_drag_Rep/NWES_start2023_09_01_end2023_10_01_RK4_B0680_tau2759_anti_beaching_cor_True_slip_velocity.zarr.

runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
B = 0.68
tau = 2994.76 #2759.97
coriolis = True
land_handling = 'anti_beaching'
starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
endtime = starttime + runtime
particle_types = ['inertial_SM_drag_Rep','inertial_drag_Rep']
data_Rep_var_1month={}
chunck_time = 100
nparticles = 88347
Rep=0
for pt in particle_types: 
    
    file = file_base.format(loc=loc,
                           particle_type = pt,
                            y_s=starttime.year,
                            m_s=starttime.month,
                            d_s=starttime.day,
                            y_e=endtime.year,
                            m_e=endtime.month,
                            d_e=endtime.day,
                            B = int(B * 1000), 
                            tau = int(tau ),
                            land_handling = land_handling, 
                            cor_on = coriolis,
                            Rep = Rep )

    ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['B','tau','z'],
                                    decode_times=False) #,decode_cf=False)

    Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    data_Rep_var_1month[pt]=ds

    

In [ ]:
# plot slip velocity
id=54634
fig, ax = plt.subplots()
# pt = 'inertial_SM_drag_Rep'
for pt, color, marker in zip(particle_types,color_array,marker_array):
    ax.plot((data_Rep_var_1month[pt].time[id,1:]-data_Rep_var_1month[pt].time[id,0])/sec_in_hours/24,data_Rep_var_1month[pt].uslip[id,1:],marker,color=color)   


ax.legend(particle_types)
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [days]')

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_var_month={}
bins_Rep_var_month={}
pdfs_Rep_var_month={}
cdfs_Rep_var_month={}
mean_Rep_var_over_month={}

for pt in particle_types:
    Re_p = Re_particle(data_Rep_var_1month[pt].Uslip,diameter, kinematic_viscosity_water)
    Re_p=np.where(Re_p>15000,15000,Re_p)
    array = Re_p.values.flatten()
    array=array[~np.isnan(array)]
    array=array[array<100000]
    Repmean =Re_p.where(Re_p<100000)
    mean_Rep_var_month[pt]=np.mean(array)
    mean_Rep_var_over_month[pt]=Repmean.mean(dim='trajectory', skipna=True)
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_var_month[pt]=bins
    pdfs_Rep_var_month[pt]=pdf
    cdfs_Rep_var_month[pt]=cdf

In [ ]:
pt = 'inertial_SM_drag_Rep'
t = 1
Re_p = Re_particle(data_Rep_var_1month[pt].Uslip[:,t],diameter, kinematic_viscosity_water)
Re_p=np.where(Re_p>15000,15000,Re_p)
Fdrag = factor_drag_white1991(Re_p)*data_Rep_var_1month[pt].Uslip[:,t]/tau
Fdrag_stokes = data_Rep_var_1month[pt].Uslip[:,t]/tau
fig, ax = plt.subplots()
ax.plot(Re_p,Fdrag_stokes,'o',markersize=8,color='navy');
ax.plot(Re_p,Fdrag,'s',markersize=8,color='firebrick');

# ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel('Re$_p$')
ax.set_ylabel('F$_{\\mathrm{drag}}$')
ax.legend(['Stokes drag','Re$_p$ dependend drag'])
ax.axvline(6250,color='grey')

In [ ]:
factor_drag_white1991(Rep)

In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,marker_array,color_array):
   
   ax.plot(bins_Rep_var_month[pt],pdfs_Rep_var_month[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')
# ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot CDF
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(particle_types,line_array,color_array):
   
   ax.plot(bins_Rep_var_month[pt],cdfs_Rep_var_month[pt],marker,color=color)


ax.legend(particle_types)
ax.set_xscale('log')


ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_SM_drag_Rep'
Re_p = Re_particle(data_Rep_var_1month[pt].Uslip,diameter, kinematic_viscosity_water)
Re_p=Re_p.ffill(dim='trajectory')
Re_p=np.where(Re_p>15000,15000,Re_p)
c_rep = factor_drag_white1991(Re_p)
tid = 350


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
# ax.coastlines()
ax.add_feature(cart.feature.LAND,color='lightgrey')
sec = float(data_Rep_var_1month[pt].time[1,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
# ax.set_title(f'{date}', fontsize=15)
# ax.set_title('15 Sep', fontsize=15)
scp = ax.scatter(data_Rep_var_1month[pt].lon[::1,tid].T,data_Rep_var_1month[pt].lat[::1,tid].T,c=Re_p[::1,tid].T,s=0.6, vmin=0,vmax=1000,cmap='Blues')
# fig.colorbar(scp,label='$\\nu_{\\mathrm{eff}}/\\nu$',fraction=0.03)
fig.colorbar(scp,label='Re$_p$',fraction=0.03)
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#         linewidth=0, color='gray', alpha=0.5, linestyle='--')
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 18}
# gl.ylabel_style =  {'size': 18}
ax.set_frame_on(False)

fig.tight_layout()
fig.savefig('../figures/poster/Rep_particle_scatter.pdf')

In [ ]:
data_Rep_var_1month[pt].time[1].values

In [ ]:
485700/3600

In [ ]:
# make movies
# make movie out of it
pt = 'inertial_drag_Rep'
Re_p = Re_particle(data_Rep_var_1month[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
for tid in range(1,719,1):

    if(tid % 100 ==0):
        print(tid)

    fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    sec = float(data_Rep_var_1month[pt].time[0,tid].values)
    date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
    ax.set_title(f'{date}', fontsize=15)
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,color='lightgrey')
    ax.set_xlim(-16,10)
    ax.set_ylim(46,61.3)
    scp = ax.scatter(data_Rep_var_1month[pt].lon[::1,tid].T,data_Rep_var_1month[pt].lat[::1,tid].T, c=Re_p[::1,tid].T,s=0.3, vmin=0,vmax=1000,cmap='magma')
    fig.colorbar(scp,label='$Re_p$',fraction=0.03)
    fig.tight_layout()
    fig.savefig(f'/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/figures/lagrangian_field/frames/september/Re_particle/1_month_hourly_field/Rep/frames{tid:04d}.png')
    plt.close(fig)

In [ ]:
# plot final positon
pt = 'inertial_drag_Rep'
Re_p = Re_particle(data_Rep_var_1month[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
tid = 700


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cart.feature.LAND,color='lightgrey')
sec = float(data_Rep_var_1month[pt].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
# ax.set_title(f'{date}', fontsize=15)
scp = ax.scatter(data_Rep_var_1month[pt].lon[::1,tid].T,data_Rep_var_1month[pt].lat[::1,tid].T,marker='o',s=2,alpha=0.1,color='firebrick')
# fig.colorbar(scp,label='$\\nu_{\\mathrm{eff}}/\\nu$',fraction=0.03)
# fig.colorbar(scp,label='Re$_p',fraction=0.03)
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

In [ ]:
fig, ax = plt.subplots()
id=1
for pt, marker, color in zip(particle_types,marker_array,color_array):
    ax.plot((data_Rep_var_1month[pt].time[id]-data_Rep_var_1month[pt].time[id,0])/sec_in_hours/24,mean_Rep_var_over_month[pt], marker, color=color )

#need small timestepping (30s) for full MR to agree with SM MR

# Measuring Rep in daily averaged field (24 h)
As Rep will on the magnitude of the velocity values in the velocity field and depend on the velocity and derivatives I also measured the slip velocity (and thus the Rep value) in daily detided field to see if this makes a big difference

In [ ]:
file = {}
# file['MR'] = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_drag_Rep/NWES_start2023_09_01_tres_daily_end2023_09_02_RK4_B0680_tau2759_anti_beaching_cor_True.zarr'
file['MRSM'] = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_SM_drag_Rep/NWES_start2023_09_01_tres_daily_end2023_10_01_RK4_B0680_tau2994_anti_beaching_cor_True.zarr'
ptypes = ['MRSM']

In [ ]:
chunck_time = 100
nparticles = 88347
ds_daily={}

for pt in ptypes:
    ds = xr.open_dataset(file[pt],
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

    Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    ds_daily[pt]=ds



In [ ]:
id=76
fig, ax = plt.subplots()

for pt, color, marker in zip(ptypes, color_array,marker_array):

    ax.plot((ds_daily[pt].time[id,1:]-ds_daily[pt].time[id,0])/sec_in_hours,ds_daily[pt].uslip[id,1:],marker,color=color)   


ax.legend(['slow manifold MR','full MR'])
ax.set_ylabel('zonal slip velocity [m/s]')
ax.set_xlabel('time [hours]')

In [ ]:
pt = 'MRSM'
Re_p = Re_particle(ds_daily[pt].Uslip,diameter, kinematic_viscosity_water)
Re_p=Re_p.ffill(dim='trajectory')
Re_p=np.where(Re_p>15000,15000,Re_p)
c_rep = factor_drag_white1991(Re_p)
tid = 350


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
# ax.coastlines()
ax.add_feature(cart.feature.LAND,color='lightgrey')
sec = float(ds_daily[pt].time[1,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
# ax.set_title(f'{date}', fontsize=15)
# ax.set_title('15 Sep', fontsize=15)
scp = ax.scatter(ds_daily[pt].lon[::1,tid].T,ds_daily[pt].lat[::1,tid].T,c=Re_p[::1,tid].T,s=0.6, vmin=0,vmax=1000,cmap='Blues')
# fig.colorbar(scp,label='$\\nu_{\\mathrm{eff}}/\\nu$',fraction=0.03)
fig.colorbar(scp,label='Re$_p$',fraction=0.03)
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
# gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
#         linewidth=0, color='gray', alpha=0.5, linestyle='--')
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 18}
# gl.ylabel_style =  {'size': 18}
ax.set_frame_on(False)

In [ ]:
# make PDF and CDF for reynoldsnumber
mean_Rep_daily_field_24h={}
bins_Rep_daily_field_24h={}
pdfs_Rep_daily_field_24h={}
cdfs_Rep_daily_field_24h={}


for pt in ptypes:
    Re_p = Re_particle(ds_daily[pt].Uslip,diameter, kinematic_viscosity_water)
    array = Re_p.values.flatten()
    array=array[~np.isnan(array)]
    array=array[array<100000]
    Repmean =Re_p.where(Re_p<100000)
    mean_Rep_daily_field_24h[pt]=np.mean(array)
    # mean_Rep_daily_field_24h[pt]=Repmean.mean(dim='trajectory', skipna=True)
    bins,pdf = make_PDF(array,nbins = 5000, norm = True)#,min=0,max=750000)
    dx=bins[1]-bins[0]
    cdf = np.cumsum(pdf*dx)
    bins_Rep_daily_field_24h[pt]=bins
    pdfs_Rep_daily_field_24h[pt]=pdf
    cdfs_Rep_daily_field_24h[pt]=cdf

In [ ]:
for pt in ptypes:
    print(mean_Rep_daily_field_24h[pt])
    print(factor_drag_morrison2013(mean_Rep_daily_field_24h[pt]))



In [ ]:
# plot pdfs
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(ptypes,marker_array,color_array):
   
   ax.plot(bins_Rep_daily_field_24h[pt],pdfs_Rep_daily_field_24h[pt],marker,color=color)


ax.legend(ptypes)
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('PDF')
ax.set_xlabel('Re$_p$')

In [ ]:
# plot CDF
fig, ax = plt.subplots()
i=0
for pt, marker, color in zip(ptypes,line_array,color_array):
   
   ax.plot(bins_Rep_daily_field_24h[pt],cdfs_Rep_daily_field_24h[pt],marker,color=color)


ax.legend(ptypes)
ax.set_xscale('log')


ax.set_ylabel('CDF')
ax.set_xlabel('Re$_p$')

In [ ]:
for pt in ptypes:
    print(mean_Rep_daily_field_24h[pt])
    print(factor_drag_white1991(mean_Rep_daily_field_24h[pt]))

In [ ]:
# scatter plot for reynolds number 
pt = 'MRSM'
Re_p = Re_particle(ds_daily[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
tid = 3


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
sec = float(ds_daily[pt].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title(f'{date}', fontsize=15)
scp = ax.scatter(ds_daily[pt].lon[::1,tid].T,ds_daily[pt].lat[::1,tid].T,c=c_rep[::1,tid].T,s=0.3, vmin=0,vmax=20)
fig.colorbar(scp,label='$\\nu_{\\mathrm{eff}}/\\nu$',fraction=0.03)
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)

## daily fields 1 month simulation varying Reynolds number

In [ ]:
file={}
file['MRSM'] = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_SM_drag_Rep/NWES_start2023_09_01_tres_daily_end2023_10_01_RK4_B0680_tau2759_anti_beaching_cor_True.zarr'
file['MR'] = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/inertial_drag_Rep/NWES_start2023_09_01_tres_daily_end2023_10_01_RK4_B0680_tau2759_anti_beaching_cor_True.zarr'
ptypes = ['MR','MRSM']

In [ ]:
ds_daily_1month={}
for pt in ptypes:
    ds = xr.open_dataset(file[pt],
                                            engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['B','tau','z'],
                                            decode_times=False) #,decode_cf=False)
    if(pt == 'MR'):
        Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2)  
    else:
        Uslip = np.sqrt(ds.uf**2 + ds.vf**2) 
    ds = ds.assign(Uslip = Uslip)
    ds.Uslip[:,0]=np.nan
    ds_daily_1month[pt]=ds

In [ ]:
pt = 'MR'
Re_p = Re_particle(ds_daily_1month[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
tid = 3


fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cart.feature.LAND,color='lightgrey')
sec = float(ds_daily_1month[pt].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title(f'{date}', fontsize=15)
scp = ax.scatter(ds_daily_1month[pt].lon[::1,tid].T,ds_daily_1month[pt].lat[::1,tid].T,c=c_rep[::1,tid].T,s=0.3, vmin=0,vmax=30,cmap='magma')
fig.colorbar(scp,label='$\\nu_{\\mathrm{eff}}/\\nu$',fraction=0.03)
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)

In [ ]:
# make movies
# make movie out of it
pt = 'MRSM'
Re_p = Re_particle(ds_daily_1month[pt].Uslip,diameter, kinematic_viscosity_water)
c_rep = factor_drag_white1991(Re_p)
for tid in range(1,719,1):

    if(tid % 100 ==0):
        print(tid)

    fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    sec = float(ds_daily_1month[pt].time[0,tid].values)
    date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
    ax.set_title(f'{date}', fontsize=15)
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,color='lightgrey')
    ax.set_xlim(-16,10)
    ax.set_ylim(46,61.3)
    scp = ax.scatter(ds_daily_1month[pt].lon[::1,tid].T,ds_daily_1month[pt].lat[::1,tid].T,c=c_rep[::1,tid].T,s=0.3, vmin=0,vmax=30,cmap='magma')
    fig.colorbar(scp,label='$Re_p$',fraction=0.03)
    fig.tight_layout()
    fig.savefig(f'/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/figures/lagrangian_field/frames/september/Re_particle/1_month_daily_field/crep/frames{tid:04d}.png')
    plt.close(fig)

In [ ]:
file_in_tracer = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/tracer/NWES_start2023_09_01_tres_daily_end2023_10_01_RK4_anti_beaching_vorticity_False.zarr'


In [ ]:
ds_tracer_daily = ds = xr.open_dataset(file_in_tracer,engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['B','tau','z'],
                                            decode_times=False) #,decode_cf=False)

In [ ]:
tid=719
fig, axs= plt.subplots(1,2,figsize=(25,12),subplot_kw={'projection':ccrs.PlateCarree()})
sec = float(ds_daily_1month[pt].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
for ax in axs:
    ax.set_title(f'{date}', fontsize=15)
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,color='lightgrey')
    ax.set_xlim(-16,0)
    ax.set_ylim(46,61.3)
# axs[0].scatter(ds_tracer_daily.lon[::1,tid].T,ds_tracer_daily.lat[::1,tid],color='orange',s=1,alpha=0.2)
axs[0].scatter(ds_daily_1month[pt].lon[::1,tid].T,ds_daily_1month[pt].lat[::1,tid],color='black',s=4,alpha=0.2)



# axs[1].scatter(ds_daily_1month[pt].lon[::1,tid].T,ds_daily_1month[pt].lat[::1,tid],color='orange',s=1,alpha=0.2)
axs[1].scatter(ds_tracer_daily.lon[::1,tid].T,ds_tracer_daily.lat[::1,tid],color='black',s=4,alpha=0.2)